<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ChemNLP_TitleToAbstract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate rouge_score jarvis-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 k

In [3]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import torch
import math
from tqdm import tqdm
import time
import evaluate
from collections import defaultdict
from jarvis.db.jsonutils import dumpjson
import random
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

random_seed = 123
torch.manual_seed(random_seed)
random.seed(0)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True

rouge_score = evaluate.load("rouge")

# import torch
# torch.cuda.is_available = lambda : False

tqdm.pandas()
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
  

In [45]:

# prompt="Nonuniform superconductivity and Josephson effect in conical ferromagnet can be described as"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
def generate_text(prompt="What is a superconductor?",max_new_tokens=250,model_checkpoint = "gpt2-medium"):
    # tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    # model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
    inputs = tokenizer(prompt, return_tensors="pt")
    # model = AutoModelForCausalLM.from_pretrained(checkpoint)
    # outputs = model.generate(**inputs, do_sample=True)
    outputs = model.generate(
        **inputs, do_sample=True, max_new_tokens=max_new_tokens
    )
    out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return (out[0].replace('\n',' '))

In [13]:
p = generate_text()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [14]:
p

'What is a superconductor?  Simply understand that while most materials absorb electromagnetic radiation, superconductor is one of the few where that radiation can be trapped inside the material—it can\'t penetrate the fabric. Instead, the electrons carry with them the electrical charge of the superconductor.  When an object is pushed against it, electrons and photons "squirm" like they could be "bent" by a thin strip of elastic tape. That charge is amplified by a magnetic field and created a wave current.  It doesn\'t do this to just anyone, though. Every superconductor contains layers upon layers of molecules of carbon, oxygen, and nitrogen. These layers form the interstitial space between the electrons, which are trapped in their quantum state and unable to escape into the electromagnetic range, but simply can\'t be affected by heat.  But don\'t worry—we don\'t want to scare you away from superconducting magnets in your home. In fact, we think they are probably more useful'

In [16]:
import os
if not os.path.exists('jarvis_leaderboard'):
  !git clone https://github.com/usnistgov/jarvis_leaderboard.git
os.chdir('jarvis_leaderboard')
!pip install -e .

Cloning into 'jarvis_leaderboard'...
remote: Enumerating objects: 37380, done.
remote: Counting objects: 100% (6180/6180), done.
remote: Compressing objects: 100% (655/655), done.
remote: Total 37380 (delta 3150), reused 5969 (delta 3079), pack-reused 31200
Receiving objects: 100% (37380/37380), 281.76 MiB | 25.82 MiB/s, done.
Resolving deltas: 100% (19318/19318), done.
Updating files: 100% (2377/2377), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/jarvis_leaderboard
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.1 MB/s eta 0:00:00
  

In [ ]:
!ls jarvis_leaderboard/benchmarks/AI/TextGen/arxiv_gen_text.json.zip

In [17]:
import json,zipfile
fname = 'jarvis_leaderboard/benchmarks/AI/TextGen/arxiv_gen_text.json.zip'
temp = 'arxiv_gen_text.json'
zp = zipfile.ZipFile(fname)
train_test = json.loads(zp.read(temp))

In [ ]:
from jarvis.db.figshare import data
import pandas as pd
arxiv_summary = data('arxiv_summary')
df = pd.DataFrame(arxiv_summary)
df = df.drop_duplicates()

In [ ]:
%%time

from tqdm import tqdm
info = {}
for i,j in tqdm(train_test['test'].items()):
  title=df[df['id']==i]['text'].values[0]
  actual_abstract =df[df['id']==i]['ctext'].values[0]
  prompt = title+' can be described as '
  #prompt = 'Write an abstract on the title '+ prompt
  pred = generate_text(prompt)
  info[i] = pred
  #break

 36%|███▌      | 35/98 [32:43<58:42, 55.92s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [44]:
info

{'1812.09136': "Nonuniform superconductivity and Josephson effect in conical ferromagnet can be described as ɛ-conductor coupled by σ c =− 1∫c = k∫x∫α b 1 /b 1, the sum of which is the superconductivity of the conical ferromagnet.  It is clear that the equation of motion is different for these two superconductor conditions. In the case of the conical ferromagnet, σ c ≈ λχ∫Ν c η ≕Φ∫ρ c /∣δ, that is, for k ≰π(∠c), the expression of σ ( σ c ) ( x ) = σ c ( x ) 2 + σ c ( x ) 0 = σ c ( x ) 2 2 + σ c ( x ) 0 0 1 + σ c ( x ) 4 3. In the case of the ferromagnet's cone, Mavericks, the equation of motion is"}

In [31]:
df[df['id']=='1812.09136']['text'].values[0]

'Nonuniform superconductivity and Josephson effect in conical ferromagnet'